### initialisation de spark

In [23]:
from pyspark.sql import SparkSession


spark = SparkSession.builder .appName("TransportProject") .getOrCreate()

### load data


In [24]:
df_bronze=spark.read.parquet('data/bronze/dataset.parquet')
df_bronze.show(5)
print(f"Nombre de lignes (Bronze) : {df_bronze.count()}")

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [25]:
from pyspark.sql.functions import col, when, count,isnan
from pyspark.sql.types import DoubleType, FloatType

def check_missing_values(df):
    exprs = []
    
    for field in df.schema.fields:
        c = field.name
        dtype = field.dataType
        
        # Numeric columns → check NULL or NaN
        if isinstance(dtype, (DoubleType, FloatType)):
            exprs.append(
                count(
                    when(col(c).isNull() | isnan(col(c)), c)
                ).alias(c)
            )
        # Other types → only NULL
        else:
            exprs.append(
                count(
                    when(col(c).isNull(), c)
                ).alias(c)
            )

    df.select(exprs).show(truncate=False)

check_missing_values(df_bronze)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|0       |0                   |0                    |540149         |0            |540149    |540149            |0           |0   

### handle missing values

In [26]:
from pyspark.sql.functions import col, unix_timestamp, hour, dayofweek, month, round
from pyspark.sql.types import (DoubleType, FloatType, IntegerType, LongType, StringType)

def handle_missing_values(df):

    # 1️⃣ NUMERIC columns (Double / Float only → mean)
    numeric_cols = [
        field.name for field in df.schema.fields
        if isinstance(field.dataType, (DoubleType, FloatType))
    ]

    for col_name in numeric_cols:
        mean_value = (
            df.select(col_name)
              .where(col(col_name).isNotNull())
              .groupBy()
              .mean()
              .first()[0]
        )

        if mean_value is not None:
            df = df.na.fill({col_name: mean_value})

    # 2️⃣ INTEGER columns (IDs, counts) → fill with -1 or 0
    int_cols = [
        field.name for field in df.schema.fields
        if isinstance(field.dataType, (IntegerType, LongType))
    ]

    for col_name in int_cols:
        df = df.na.fill({col_name: -1})

    # 3️⃣ STRING columns ONLY → fill with "Unknown"
    string_cols = [
        field.name for field in df.schema.fields
        if isinstance(field.dataType, StringType)
    ]

    for col_name in string_cols:
        df = df.na.fill({col_name: "Unknown"})

    # 4️⃣ DO NOTHING for TIMESTAMP columns (very important)
    return df


df_cleaned = handle_missing_values(df_bronze)

### handle duplicates

In [27]:
def handle_duplicates(df):
    return df.dropDuplicates()

df_no_duplicates = handle_duplicates(df_cleaned)

### Silver Layer

In [28]:
from pyspark.sql.functions import (
    col, unix_timestamp, hour, dayofweek, month, round
)

# 🔹 Sécurité : supprimer les colonnes si elles existent déjà
cols_to_drop = ["duration_minutes", "pickuphour", "dayof_week", "month"]
df_silver = df_no_duplicates.drop(*[c for c in cols_to_drop if c in df_no_duplicates.columns])

# 1️⃣ Calcul de la durée en minutes
df_silver = df_silver.withColumn(
    "duration_minutes",
    round(
        (unix_timestamp("tpep_dropoff_datetime") -
         unix_timestamp("tpep_pickup_datetime")) / 60,
        2
    )
)

# 2️⃣ Filtrage des trajets aberrants
df_silver = df_silver.filter(
    (col("trip_distance") > 0) &
    (col("trip_distance") <= 200) &
    (col("duration_minutes") > 0) &
    (col("passenger_count") > 0)
)

# 3️⃣ Features temporelles
df_silver = (
    df_silver
    .withColumn("pickuphour", hour("tpep_pickup_datetime"))
    .withColumn("dayof_week", dayofweek("tpep_pickup_datetime"))
    .withColumn("month", month("tpep_pickup_datetime"))
)

# 4️⃣ Vérification
df_silver.show(5)
df_silver.printSchema()


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+----------------+----------+----------+-----+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|duration_minutes|pickuphour|dayof_week|month|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+----------------+----------+----------+--

In [29]:
df_silver.select(
    "trip_distance",
    "duration_minutes",
    "passenger_count",
    "pickuphour",
    "dayof_week",
    "month"
).summary().show()


+-------+------------------+------------------+------------------+------------------+------------------+--------------------+
|summary|     trip_distance|  duration_minutes|   passenger_count|        pickuphour|        dayof_week|               month|
+-------+------------------+------------------+------------------+------------------+------------------+--------------------+
|  count|           2871275|           2871275|           2871275|           2871275|           2871275|             2871275|
|   mean|3.1930232631845143|15.111674618418885|1.3080283149471925|14.387960749144543| 4.247460448755344|  1.0000808003413117|
| stddev| 4.359738082143162|29.095096636594906|0.7430467194763688| 5.534282505920479|1.8933577854610746|0.029754239374199302|
|    min|              0.01|              0.02|                 1|                 0|                 1|                   1|
|    25%|              0.99|              7.05|                 1|                11|                 3|              

### Stockage Silver 

In [ ]:

df_silver.write \
    .mode("overwrite") \
    .parquet("data/silver")


26/01/07 10:11:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
